In [1]:
import numpy as np
import pydotplus
import pandas as pd
import sklearn.datasets as Datasets
from sklearn import tree
from sklearn import model_selection as cv

# Creating Decision Tree class to use tree for prediction

In [2]:
class DecisionTree(object):
    def __init__(self):
        self.children = {} #If size = 0, implies it is leaf node. Hash is to keep track that which child node corresponds to which label
        self.split_feature = None
        self.output_class = None # Required when we reach leaf or pure node

### Utility Function

In [3]:
def makeLabelled(column):
    mean = column.mean()
    for i in range (0,len(column)):
        column[i] = int(column[i]>=mean) 
    return column

# Modifying Iris Dataset to change it to Labeled Data

In [4]:
iris = Datasets.load_iris()
df = pd.DataFrame(iris.data)

In [5]:
X = df.values
Y = iris.target

In [6]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [7]:
X_train, X_test, Y_train, Y_test = cv.train_test_split(X,Y)

# Testing results using sklearn decision tree

In [8]:
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
dot_data = tree.export_graphviz(clf,out_file= None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("irisDecisionTree.pdf")

True

In [10]:
clf.score(X_test,Y_test)

0.73684210526315785

# My Implementation

In [11]:
def entropy(Y):
    classes = set(Y)
    value = 0
    for i in classes:
        p = (len(Y[Y==i])/len(Y))
        value = value - (p*np.log2(p))
    return value

In [12]:
def findGainRatio(X,Y,selectedFeature):
    differentLabels = set(X[:,selectedFeature])
    entropyBeforeSplitting = entropy(Y)
    entropyAfterSplitting = 0
    splitInfo = 0
    for i in differentLabels:
        newNodeY = Y[(X[:,selectedFeature] == i)]
        weightOfSamples = (len(newNodeY)/len(Y))
        entropyAfterSplitting = entropyAfterSplitting + (weightOfSamples*entropy(newNodeY))
        splitInfo = splitInfo - (weightOfSamples*np.log2(weightOfSamples))
    #print("Entropy Before Splitting = ",entropyBeforeSplitting)
    #print("Entropy After Splitting = ",entropyAfterSplitting)
    gain = entropyBeforeSplitting - entropyAfterSplitting
    gainRatio = gain#/splitInfo
    #print("gain is = ",gain)
    #print("split info is = ",splitInfo)
    #print("gain Ratio is = ",gainRatio)
    return gainRatio    

In [13]:
def findMajorityClass(Y):
    classes = set(Y)
    ans = -1
    max_value = -1
    for i in classes:
        if(len(Y[Y==i]) >= max_value):
            ans = i
            max_value = len(Y[Y==i])
    return ans

### Utility functions for printing only

In [14]:
def printClassification(Y):
    classes = set(Y)
    for i in classes:
        print("Count of ",i," = ",len(Y[Y==i]))

In [15]:
def printDTree(dTree):
    if(len(dTree.children.keys()) == 0):
        print("leaf node with output = ",dTree.output_class)
        return
    print("split on feature = ", dTree.split_feature)
    for i in dTree.children.keys():
        printDTree(dTree.children[i])

In [16]:
def printSplitsInDFS(X,Y,available_features,used_features=[]):
    print(" ")
    printClassification(Y)
    print("Current Entropy is = ",entropy(Y))
    if(available_features == 0 or (entropy(Y) == 0)):
        print("Reached leaf Node having majority class = ",findMajorityClass(Y))
        return
    selectedFeature = -1 #representing no feature is selected yet
    max_value = -float('inf')
    for i in range(0,X.shape[-1]):
        if(i in used_features):
            continue
        value = findGainRatio(X,Y,i)
        if(value >= max_value and value >= 0):
            selectedFeature = i
            max_value = value
    if(selectedFeature == -1): 
        return # as no feature can be used to split
    
    #If after splitting about selected feature, all samples go to one node only, then splitting is of no use and mark it as leaf node
    differentLabels = set(X[:,selectedFeature])
    if(len(differentLabels) == 1):
        print("Reached leaf Node having majority class = ",findMajorityClass(Y))
        return
    
    print("Splitting on feature ",selectedFeature)
    used_features.append(selectedFeature)
    for i in differentLabels:
        newDataSamplesIndex = (X[:,selectedFeature] == i)
        newX = X[newDataSamplesIndex]
        newY = Y[newDataSamplesIndex]
        printSplitsInDFS(newX,newY,available_features-1,used_features)
    used_features.remove(selectedFeature) # As this feature can be used again in other branches for splitting
    return

## Main methods for fitting and predicting

In [17]:
def fitDecisionTree(X,Y,available_features,used_features=[]):
    dTree = DecisionTree()
    #Save output class on basis of majority
    dTree.output_class = findMajorityClass(Y)
    
    if(available_features == 0 or (entropy(Y) == 0)):
        return dTree
    #Finding feature about which we want to split
    selectedFeature = -1 #representing no feature is selected yet
    max_value = -float('inf')
    for i in range(0,X.shape[-1]):
        if(i in used_features):
            continue
        value = findGainRatio(X,Y,i)
        if(value >= max_value and value >= 0):
            selectedFeature = i
            max_value = value
    if(selectedFeature == -1):  #Implies this node will be a leaf node as no feature can be used to split
        return dTree
    
    #If after splitting about selected feature, all samples go to one node only, then splitting is of no use and mark it as leaf node
    differentLabels = set(X[:,selectedFeature])
    if(len(differentLabels) == 1):
        return dTree
    
    used_features.append(selectedFeature) #Marking this feature as used so that it can't be used again in any of its subtrees for splitting.
    dTree.split_feature = selectedFeature
    
    #For each unique value of selected feature, add a node for it.
    for i in differentLabels: #If labels are 1,2,3... then children dictionary for key 1, will store corrsponding child subtree
        newDataSamplesIndex = (X[:,selectedFeature] == i) #Boolean array
        newX = X[newDataSamplesIndex]
        newY = Y[newDataSamplesIndex]
        dTree.children[i] = fitDecisionTree(newX,newY,available_features-1,used_features)
    used_features.remove(selectedFeature) # As this feature can be used again in other subtrees of its parent for splitting
    return dTree

In [25]:
#Steps:
#1. Check feature to split about using dTree
#2. If label of that feature is 'i', then go to corresponding child of that node.
#3. Do this recursively until you reach a leaf node.

def predictForOneSample(dTree,X_test_sample):
    if(len(dTree.children.keys()) == 0):
        return dTree.output_class
    label = X_test_sample[dTree.split_feature] # Label of feature of test sample about which DTree tells us to take decision
    return predictForOneSample(dTree.children[label],X_test_sample)
    
def predictUsingDecisionTree(dTree,X_test):
    Y_pred = np.zeros(X_test.shape[0])
    for i in range(0,X_test.shape[0]): #Predict for each sample one by one
        Y_pred[i] = predictForOneSample(dTree,X_test[i,:])
    return Y_pred

# Trying on iris Dataset

In [19]:
dTree = fitDecisionTree(X_train,Y_train,X_train.shape[-1])

In [30]:
Y_pred = predictUsingDecisionTree(dTree,X_test)
print("Score is = ",(Y_pred == Y_test).sum()/len(Y_test))

Score is =  0.736842105263


# XOR logic in DFS Manner

In [22]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0,1,1,0])
printSplitsInDFS(X,Y,2)

 
Count of  0  =  2
Count of  1  =  2
Current Entropy is =  1.0
Splitting on feature  1
 
Count of  0  =  1
Count of  1  =  1
Current Entropy is =  1.0
Splitting on feature  0
 
Count of  0  =  1
Current Entropy is =  0.0
Reached leaf Node having majority class =  0
 
Count of  1  =  1
Current Entropy is =  0.0
Reached leaf Node having majority class =  1
 
Count of  0  =  1
Count of  1  =  1
Current Entropy is =  1.0
Splitting on feature  0
 
Count of  1  =  1
Current Entropy is =  0.0
Reached leaf Node having majority class =  1
 
Count of  0  =  1
Current Entropy is =  0.0
Reached leaf Node having majority class =  0
